# Part III: Core FastAPI Concepts

## Chapter 6: Dependency Injection System

Dependency injection is one of FastAPI's most powerful and distinctive features. It provides a clean, declarative way to share logic across your application, handle common operations like database connections and authentication, and make your code more testable and maintainable. This chapter will teach you everything you need to know about mastering FastAPI's dependency injection system.

---

### Understanding Dependency Injection

**Dependency injection (DI)** is a design pattern where a class or function receives its dependencies from an external source rather than creating them internally. In FastAPI, this means your path operation functions can declare what they need (dependencies), and FastAPI will provide those dependencies automatically.

#### Why Dependency Injection Matters

Consider a traditional approach where each endpoint handles its own setup:

```python
# ❌ WITHOUT dependency injection - repetitive and error-prone
from fastapi import FastAPI, HTTPException, Header

app = FastAPI()


def get_current_user(authorization: str | None = Header(default=None)):
    """Extract and validate user from auth header."""
    if authorization is None:
        raise HTTPException(status_code=401, detail="Not authenticated")

    # Validate token
    token = authorization.replace("Bearer ", "")
    user = validate_token(token)  # Assume this exists
    if not user:
        raise HTTPException(status_code=401, detail="Invalid token")

    return user


@app.get("/users/me")
async def get_my_profile(authorization: str | None = Header(default=None)):
    user = get_current_user(authorization)
    return {"user_id": user.id, "username": user.username}


@app.get("/users/me/posts")
async def get_my_posts(authorization: str | None = Header(default=None)):
    user = get_current_user(authorization)
    posts = get_posts_for_user(user.id)  # Assume this exists
    return {"posts": posts}


@app.delete("/users/me")
async def delete_my_account(authorization: str | None = Header(default=None)):
    user = get_current_user(authorization)
    delete_user(user.id)  # Assume this exists
    return {"message": "Account deleted"}
```

With dependency injection, this becomes much cleaner:

```python
# ✅ WITH dependency injection - clean and reusable
from fastapi import FastAPI, Depends, HTTPException, Header

app = FastAPI()


async def get_current_user(authorization: str | None = Header(default=None)):
    """Dependency that extracts and validates user from auth header."""
    if authorization is None:
        raise HTTPException(status_code=401, detail="Not authenticated")

    token = authorization.replace("Bearer ", "")
    user = validate_token(token)
    if not user:
        raise HTTPException(status_code=401, detail="Invalid token")

    return user


@app.get("/users/me")
async def get_my_profile(user = Depends(get_current_user)):
    return {"user_id": user.id, "username": user.username}


@app.get("/users/me/posts")
async def get_my_posts(user = Depends(get_current_user)):
    posts = get_posts_for_user(user.id)
    return {"posts": posts}


@app.delete("/users/me")
async def delete_my_account(user = Depends(get_current_user)):
    delete_user(user.id)
    return {"message": "Account deleted"}
```

**Benefits of dependency injection:**
- **Reusability**: Write logic once, use it everywhere
- **Testability**: Easily swap dependencies with mocks
- **Clean code**: Path operations focus on business logic
- **Automatic documentation**: Dependencies appear in OpenAPI schema
- **Automatic validation**: Dependencies can validate and raise HTTP errors

---

### 6.1 The `Depends` Function: Creating Reusable Logic

The `Depends` function is the heart of FastAPI's dependency injection system. It tells FastAPI to call a function and use its return value as a parameter.

#### Basic Dependency

```python
from fastapi import FastAPI, Depends

app = FastAPI()


def common_parameters(q: str | None = None, skip: int = 0, limit: int = 100):
    """
    A dependency that provides common query parameters.
    This function is called automatically by FastAPI.
    """
    return {"q": q, "skip": skip, "limit": limit}


@app.get("/items")
async def read_items(commons: dict = Depends(common_parameters)):
    """
    The 'commons' parameter receives the return value from common_parameters.
    """
    return commons


@app.get("/users")
async def read_users(commons: dict = Depends(common_parameters)):
    """Same dependency used in multiple endpoints."""
    return commons
```

**Request flow:**
1. Client sends `GET /items?q=laptop&skip=10&limit=50`
2. FastAPI calls `common_parameters(q="laptop", skip=10, limit=50)`
3. `common_parameters` returns `{"q": "laptop", "skip": 10, "limit": 50}`
4. `read_items` receives this dict as the `commons` parameter

#### Dependencies as Classes

You can use classes as dependencies, which is often cleaner for grouping related parameters:

```python
from fastapi import FastAPI, Depends, Query
from typing import Annotated

app = FastAPI()


class CommonParams:
    """A class-based dependency for common query parameters."""

    def __init__(
        self,
        q: Annotated[str | None, Query(max_length=50)] = None,
        skip: int = Query(default=0, ge=0),
        limit: int = Query(default=100, le=100),
    ):
        self.q = q
        self.skip = skip
        self.limit = limit


@app.get("/items")
async def read_items(commons: CommonParams = Depends(CommonParams)):
    """
    FastAPI creates a new CommonParams instance for each request.
    """
    response = {"skip": commons.skip, "limit": commons.limit}
    if commons.q:
        response["q"] = commons.q
    return response


@app.get("/users")
async def read_users(commons: CommonParams = Depends(CommonParams)):
    """Same dependency, different endpoint."""
    return {"skip": commons.skip, "limit": commons.limit}
```

#### Annotated Dependencies

The `Annotated` pattern is the modern, recommended approach:

```python
from typing import Annotated
from fastapi import FastAPI, Depends, Query

app = FastAPI()


class Pagination:
    def __init__(
        self,
        skip: int = Query(default=0, ge=0),
        limit: int = Query(default=20, le=100),
    ):
        self.skip = skip
        self.limit = limit


# Create a reusable type alias
PaginationDep = Annotated[Pagination, Depends(Pagination)]


@app.get("/items")
async def read_items(pagination: PaginationDep):
    """Using the Annotated dependency type."""
    return {"skip": pagination.skip, "limit": pagination.limit}


@app.get("/users")
async def read_users(pagination: PaginationDep):
    """Same dependency, cleaner syntax."""
    return {"skip": pagination.skip, "limit": pagination.limit}
```

---

> **Industry Standard:** Use `Annotated` for dependencies. It keeps your function signatures clean and enables better IDE support and type checking.

---

#### Dependency with Path Parameters

Dependencies can access path parameters, query parameters, and request body just like path operations:

```python
from fastapi import FastAPI, Depends, Path, Query
from typing import Annotated

app = FastAPI()


def validate_item_id(item_id: int = Path(gt=0, description="The item ID")):
    """Dependency that validates the item ID path parameter."""
    # Could add additional validation or database lookup
    return item_id


def check_access(
    user_id: Annotated[int, Query(description="User ID for access check")],
    item_id: int = Path(),
):
    """Dependency that checks if user can access an item."""
    # Business logic for access control
    if user_id == 1:  # Simplified example
        return {"user_id": user_id, "item_id": item_id, "access": True}
    raise HTTPException(status_code=403, detail="Access denied")


@app.get("/items/{item_id}")
async def read_item(
    item_id: int = Depends(validate_item_id),
    access: dict = Depends(check_access),
):
    """Multiple dependencies on a single endpoint."""
    return {"item_id": item_id, "access": access}
```

#### Dependencies with Database Sessions

A common pattern is using dependencies for database sessions:

```python
from fastapi import FastAPI, Depends, HTTPException
from pydantic import BaseModel
from typing import Annotated

app = FastAPI()


# Simulated database
class Database:
    def __init__(self):
        self.items = {
            1: {"id": 1, "name": "Laptop", "price": 999.99},
            2: {"id": 2, "name": "Mouse", "price": 29.99},
        }
        self._connected = True

    def is_connected(self):
        return self._connected

    def get_item(self, item_id: int):
        return self.items.get(item_id)

    def close(self):
        self._connected = False
        print("Database connection closed")


# Simulated database session
def get_db():
    """Dependency that provides a database session."""
    db = Database()
    try:
        yield db
    finally:
        db.close()


# Simulated models
class Item(BaseModel):
    id: int
    name: str
    price: float


# Type alias for dependency
DBDep = Annotated[Database, Depends(get_db)]


@app.get("/items/{item_id}", response_model=Item)
async def get_item(item_id: int, db: DBDep):
    """
    Get an item by ID.

    The database session is automatically provided by the dependency.
    """
    item = db.get_item(item_id)
    if item is None:
        raise HTTPException(status_code=404, detail="Item not found")
    return item


@app.get("/items")
async def list_items(db: DBDep):
    """List all items."""
    return {"items": list(db.items.values()), "connected": db.is_connected()}
```

#### Global Dependencies

Apply dependencies to all endpoints in an application or router:

```python
from fastapi import FastAPI, Depends, Header, HTTPException
from typing import Annotated

app = FastAPI()


async def verify_token(x_token: Annotated[str, Header()]):
    """Verify authentication token from header."""
    if x_token != "fake-super-secret-token":
        raise HTTPException(status_code=400, detail="X-Token header invalid")


async def verify_key(x_key: Annotated[str, Header()]):
    """Verify API key from header."""
    if x_key != "fake-super-secret-key":
        raise HTTPException(status_code=400, detail="X-Key header invalid")


# Apply dependencies globally to all routes
app = FastAPI(dependencies=[Depends(verify_token), Depends(verify_key)])


@app.get("/items")
async def read_items():
    """Both verify_token and verify_key are called before this."""
    return [{"item": "Foo"}, {"item": "Bar"}]


@app.get("/users")
async def read_users():
    """Dependencies apply here too."""
    return [{"username": "Rick"}, {"username": "Morty"}]
```

---

### 6.2 Dependency Trees: Dependencies Calling Other Dependencies

One of the most powerful features of FastAPI's DI system is that dependencies can themselves have dependencies. This creates a **dependency tree** where FastAPI resolves all dependencies in the correct order.

#### Basic Dependency Tree

```python
from fastapi import FastAPI, Depends, HTTPException, Header
from typing import Annotated

app = FastAPI()


def extract_token(authorization: Annotated[str, Header()]):
    """First-level dependency: extract token from header."""
    if not authorization.startswith("Bearer "):
        raise HTTPException(status_code=401, detail="Invalid authorization header")
    return authorization.replace("Bearer ", "")


def validate_token(token: str = Depends(extract_token)):
    """Second-level dependency: validate the extracted token."""
    # Simulated token validation
    valid_tokens = {"secret-token-123": {"user_id": 1, "username": "alice"}}

    if token not in valid_tokens:
        raise HTTPException(status_code=401, detail="Invalid token")

    return valid_tokens[token]


def get_current_user(user_data: dict = Depends(validate_token)):
    """Third-level dependency: get user from validated token."""
    # Could query database for full user data
    return {"id": user_data["user_id"], "username": user_data["username"]}


# Type alias
UserDep = Annotated[dict, Depends(get_current_user)]


@app.get("/users/me")
async def get_my_profile(user: UserDep):
    """
    The dependency tree is:
    extract_token -> validate_token -> get_current_user -> this function
    """
    return {"user": user}
```

**Execution flow:**

```
Request: GET /users/me
Headers: Authorization: Bearer secret-token-123

↓
1. extract_token(authorization="Bearer secret-token-123")
   → returns "secret-token-123"
↓
2. validate_token(token="secret-token-123")
   → returns {"user_id": 1, "username": "alice"}
↓
3. get_current_user(user_data={"user_id": 1, "username": "alice"})
   → returns {"id": 1, "username": "alice"}
↓
4. get_my_profile(user={"id": 1, "username": "alice"})
   → returns {"user": {"id": 1, "username": "alice"}}
```

#### Real-World Authentication Example

Let's build a complete authentication system using dependency trees:

```python
from datetime import datetime, timedelta, timezone
from typing import Annotated, Any

from fastapi import FastAPI, Depends, HTTPException, Header, status
from fastapi.security import OAuth2PasswordBearer
from pydantic import BaseModel
import jwt  # PyJWT library

app = FastAPI()

# Configuration
SECRET_KEY = "your-secret-key-keep-it-secret"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

# Simulated database
fake_users_db = {
    "alice": {
        "id": 1,
        "username": "alice",
        "email": "alice@example.com",
        "hashed_password": "fakehashedsecret",
        "is_active": True,
        "is_superuser": False,
    },
    "admin": {
        "id": 2,
        "username": "admin",
        "email": "admin@example.com",
        "hashed_password": "fakehashedadmin",
        "is_active": True,
        "is_superuser": True,
    },
}


# Models
class Token(BaseModel):
    access_token: str
    token_type: str


class User(BaseModel):
    id: int
    username: str
    email: str
    is_active: bool = True
    is_superuser: bool = False


# OAuth2 scheme for Swagger UI
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token", auto_error=False)


# Dependencies
async def get_token(token: str | None = Depends(oauth2_scheme)):
    """Extract token from Authorization header."""
    if token is None:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Not authenticated",
            headers={"WWW-Authenticate": "Bearer"},
        )
    return token


async def decode_token(token: str = Depends(get_token)) -> dict[str, Any]:
    """Decode and validate JWT token."""
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        return payload
    except jwt.ExpiredSignatureError:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Token has expired",
            headers={"WWW-Authenticate": "Bearer"},
        )
    except jwt.InvalidTokenError:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Invalid token",
            headers={"WWW-Authenticate": "Bearer"},
        )


async def get_current_user(payload: dict = Depends(decode_token)) -> User:
    """Get the current user from token payload."""
    username: str = payload.get("sub")
    if username is None:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Invalid token payload",
        )

    user_data = fake_users_db.get(username)
    if user_data is None:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="User not found",
        )

    return User(**user_data)


async def get_current_active_user(user: User = Depends(get_current_user)) -> User:
    """Ensure the user is active."""
    if not user.is_active:
        raise HTTPException(status_code=400, detail="Inactive user")
    return user


async def get_current_superuser(user: User = Depends(get_current_active_user)) -> User:
    """Ensure the user is a superuser."""
    if not user.is_superuser:
        raise HTTPException(
            status_code=status.HTTP_403_FORBIDDEN,
            detail="Not enough permissions",
        )
    return user


# Type aliases for cleaner signatures
UserDep = Annotated[User, Depends(get_current_active_user)]
SuperuserDep = Annotated[User, Depends(get_current_superuser)]


# Token generation
def create_access_token(data: dict, expires_delta: timedelta | None = None):
    """Create a JWT access token."""
    to_encode = data.copy()
    if expires_delta:
        expire = datetime.now(timezone.utc) + expires_delta
    else:
        expire = datetime.now(timezone.utc) + timedelta(minutes=15)
    to_encode.update({"exp": expire})
    encoded_jwt = jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
    return encoded_jwt


# Endpoints
@app.post("/token", response_model=Token)
async def login(username: str, password: str):
    """Generate an access token (simplified - no password verification)."""
    user = fake_users_db.get(username)
    if not user:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Incorrect username or password",
        )

    access_token = create_access_token(
        data={"sub": user["username"]},
        expires_delta=timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES),
    )
    return {"access_token": access_token, "token_type": "bearer"}


@app.get("/users/me", response_model=User)
async def read_users_me(user: UserDep):
    """Get current user profile."""
    return user


@app.get("/users/me/items")
async def read_own_items(user: UserDep):
    """Get items belonging to current user."""
    return [{"item_id": 1, "owner": user.username}]


@app.get("/admin/users")
async def list_all_users(admin: SuperuserDep):
    """List all users - admin only."""
    return {"users": list(fake_users_db.values()), "requested_by": admin.username}
```

**Dependency Tree Visualization:**

```
get_token (extracts token from header)
    ↓
decode_token (validates JWT)
    ↓
get_current_user (fetches user from DB)
    ↓
get_current_active_user (checks if active)
    ↓
get_current_superuser (checks if admin)  ← Optional, for admin routes
    ↓
Endpoint handler
```

#### Combining Multiple Dependencies

An endpoint can have multiple independent dependencies:

```python
from fastapi import FastAPI, Depends, Query, Header
from typing import Annotated

app = FastAPI()


class Pagination:
    def __init__(self, skip: int = 0, limit: int = 100):
        self.skip = skip
        self.limit = limit


def get_pagination(skip: int = 0, limit: int = 100) -> Pagination:
    return Pagination(skip=skip, limit=limit)


def get_api_version(x_api_version: Annotated[str, Header()] = "v1"):
    return x_api_version


def get_request_id(x_request_id: Annotated[str, Header()] = ""):
    return x_request_id or "default-request-id"


# Type aliases
PaginationDep = Annotated[Pagination, Depends(get_pagination)]
VersionDep = Annotated[str, Depends(get_api_version)]
RequestIdDep = Annotated[str, Depends(get_request_id)]


@app.get("/items")
async def list_items(
    pagination: PaginationDep,
    api_version: VersionDep,
    request_id: RequestIdDep,
):
    """
    Multiple independent dependencies:
    - Pagination: handles skip/limit
    - api_version: from header
    - request_id: from header
    """
    return {
        "skip": pagination.skip,
        "limit": pagination.limit,
        "api_version": api_version,
        "request_id": request_id,
    }
```

---

### 6.3 Yield Dependencies: Setup and Teardown Logic

Dependencies can perform setup and teardown operations using Python generators with `yield`. This is essential for managing resources like database connections, file handles, or any resource that needs cleanup.

#### Basic Yield Dependency

```python
from fastapi import FastAPI, Depends

app = FastAPI()


def get_db():
    """
    Yield dependency for database session.
    Code before yield runs before the endpoint.
    Code after yield runs after the endpoint completes.
    """
    # Setup: Create database connection
    print("Opening database connection")
    db = {"connection": "active", "items": []}
    
    try:
        yield db  # This is passed to the endpoint
    finally:
        # Teardown: Always runs, even if an exception occurs
        print("Closing database connection")
        db["connection"] = "closed"


@app.get("/items")
async def get_items(db = Depends(get_db)):
    print("Processing request")
    return {"items": db["items"], "status": db["connection"]}


@app.post("/items")
async def create_item(item: str, db = Depends(get_db)):
    print("Creating item")
    db["items"].append(item)
    return {"created": item}
```

**Request flow for `GET /items`:**

```text
1. get_db() starts
2. Setup code runs: "Opening database connection"
3. yield db - db is passed to endpoint
4. Endpoint runs: "Processing request"
5. Endpoint returns
6. Finally block runs: "Closing database connection"
```

#### Async Yield Dependencies

For async resources, use `async def` with `yield`:

```python
from fastapi import FastAPI, Depends
from typing import AsyncGenerator

app = FastAPI()


async def get_async_db() -> AsyncGenerator[dict, None]:
    """
    Async yield dependency for database sessions.
    Use async/await for async resources.
    """
    # Setup
    print("Setting up async database connection")
    db = {"connection": "active", "pool": "connected"}
    
    try:
        yield db
    finally:
        # Teardown
        print("Tearing down async database connection")
        db["connection"] = "closed"


# Type alias
from typing import Annotated
DBDep = Annotated[dict, Depends(get_async_db)]


@app.get("/data")
async def get_data(db: DBDep):
    """Endpoint using async database connection."""
    return {"status": db["connection"]}
```

#### Database Session Pattern

A realistic pattern for SQLAlchemy-like database sessions:

```python
from typing import Annotated, Generator
from fastapi import FastAPI, Depends, HTTPException
from pydantic import BaseModel
from contextlib import contextmanager

app = FastAPI()


# Simulated database session class
class DatabaseSession:
    def __init__(self, connection_string: str):
        self.connection_string = connection_string
        self._is_active = False
        self._data = {
            1: {"id": 1, "name": "Laptop", "price": 999.99},
            2: {"id": 2, "name": "Mouse", "price": 29.99},
        }
    
    def connect(self):
        """Establish connection."""
        print(f"Connecting to {self.connection_string}")
        self._is_active = True
    
    def close(self):
        """Close connection."""
        print("Closing database connection")
        self._is_active = False
    
    def get(self, item_id: int):
        """Get item by ID."""
        return self._data.get(item_id)
    
    def add(self, item: dict):
        """Add new item."""
        new_id = max(self._data.keys()) + 1
        item["id"] = new_id
        self._data[new_id] = item
        return item
    
    def commit(self):
        """Commit transaction."""
        print("Committing transaction")
    
    def rollback(self):
        """Rollback transaction."""
        print("Rolling back transaction")
    
    def is_active(self):
        return self._is_active


def get_db() -> Generator[DatabaseSession, None, None]:
    """
    Dependency that provides a database session.
    Handles connection, transaction, and cleanup.
    """
    # Setup: Create and connect
    db = DatabaseSession("postgresql://localhost/app_db")
    db.connect()
    
    try:
        yield db
        # If we reach here, the endpoint succeeded
        db.commit()
    except Exception as e:
        # Something went wrong, rollback
        db.rollback()
        raise  # Re-raise the exception
    finally:
        # Always close the connection
        db.close()


# Type alias
DBDep = Annotated[DatabaseSession, Depends(get_db)]


# Models
class ItemCreate(BaseModel):
    name: str
    price: float


class Item(BaseModel):
    id: int
    name: str
    price: float


# Endpoints
@app.get("/items/{item_id}", response_model=Item)
async def get_item(item_id: int, db: DBDep):
    """Get an item by ID."""
    item = db.get(item_id)
    if item is None:
        raise HTTPException(status_code=404, detail="Item not found")
    return item


@app.post("/items", response_model=Item, status_code=201)
async def create_item(item: ItemCreate, db: DBDep):
    """Create a new item."""
    new_item = db.add(item.model_dump())
    return new_item


@app.get("/items")
async def list_items(db: DBDep, skip: int = 0, limit: int = 10):
    """List all items with pagination."""
    items = list(db._data.values())[skip : skip + limit]
    return {"items": items, "db_active": db.is_active()}
```

#### Yield Dependencies with Error Handling

```python
from fastapi import FastAPI, Depends, HTTPException
from typing import Annotated, Generator

app = FastAPI()


class ServiceClient:
    """Simulated external service client."""

    def __init__(self, service_name: str):
        self.service_name = service_name
        self._is_connected = False
    
    def connect(self):
        print(f"Connecting to {self.service_name}...")
        self._is_connected = True
        print(f"Connected to {self.service_name}")
    
    def disconnect(self):
        print(f"Disconnecting from {self.service_name}...")
        self._is_connected = False
    
    def call(self, method: str) -> str:
        if not self._is_connected:
            raise RuntimeError("Not connected")
        return f"Result from {self.service_name}.{method}"


def get_service_client() -> Generator[ServiceClient, None, None]:
    """
    Yield dependency with comprehensive error handling.
    """
    client = ServiceClient("payment-gateway")
    
    # Setup
    try:
        client.connect()
    except Exception as e:
        raise HTTPException(
            status_code=503,
            detail=f"Failed to connect to service: {str(e)}"
        )
    
    try:
        yield client
    except HTTPException:
        # Re-raise HTTP exceptions without modification
        raise
    except Exception as e:
        # Convert unexpected errors to HTTP errors
        raise HTTPException(
            status_code=500,
            detail=f"Service error: {str(e)}"
        )
    finally:
        # Always cleanup
        client.disconnect()


ServiceDep = Annotated[ServiceClient, Depends(get_service_client)]


@app.post("/payments")
async def process_payment(service: ServiceDep, amount: float):
    """Process a payment using the service client."""
    if amount <= 0:
        raise HTTPException(status_code=400, detail="Amount must be positive")
    
    result = service.call("process_payment")
    return {"result": result, "amount": amount}
```

#### Multiple Yield Dependencies

You can have multiple yield dependencies, and they're cleaned up in reverse order:

```python
from fastapi import FastAPI, Depends
from typing import Annotated, Generator

app = FastAPI()


def get_db():
    print("1. DB Setup")
    try:
        yield "database"
    finally:
        print("5. DB Teardown")


def get_cache():
    print("2. Cache Setup")
    try:
        yield "cache"
    finally:
        print("4. Cache Teardown")


def get_logger():
    print("3. Logger Setup")
    try:
        yield "logger"
    finally:
        print("6. Logger Teardown")


DBDep = Annotated[str, Depends(get_db)]
CacheDep = Annotated[str, Depends(get_cache)]
LoggerDep = Annotated[str, Depends(get_logger)]


@app.get("/test")
async def test_endpoint(db: DBDep, cache: CacheDep, logger: LoggerDep):
    print("=== Processing request ===")
    return {"db": db, "cache": cache, "logger": logger}
```

**Output:**

```text
1. DB Setup
2. Cache Setup
3. Logger Setup
=== Processing request ===
6. Logger Teardown
4. Cache Teardown
5. DB Teardown
```

---

> **Important:** Dependencies are initialized in the order they appear in the function signature, but cleaned up in **reverse order** (LIFO - Last In, First Out). This ensures proper resource cleanup.

---

### 6.4 Overrides: Testing and Mocking Dependencies Efficiently

One of the greatest benefits of dependency injection is the ability to **override dependencies** during testing. This allows you to substitute real dependencies with mocks, stubs, or test-specific implementations without changing your application code.

#### Why Override Dependencies?

In testing, you often need to:
- Replace a database with an in-memory test database
- Mock external API calls
- Use test users instead of real authentication
- Simulate error conditions

#### Basic Dependency Override

```python
# main.py
from fastapi import FastAPI, Depends, HTTPException, Header
from typing import Annotated

app = FastAPI()


def get_current_user(authorization: Annotated[str, Header()]):
    """Production dependency - validates real tokens."""
    if not authorization.startswith("Bearer "):
        raise HTTPException(status_code=401, detail="Invalid token")
    # Would validate token and fetch user from database
    return {"id": 1, "username": "real_user"}


UserDep = Annotated[dict, Depends(get_current_user)]


@app.get("/users/me")
async def get_my_profile(user: UserDep):
    return {"user": user}
```

```python
# test_main.py
from fastapi.testclient import TestClient
from main import app, get_current_user


def override_get_current_user():
    """Test dependency - returns a mock user."""
    return {"id": 99, "username": "test_user", "is_test": True}


# Create test client
client = TestClient(app)


def test_get_profile_with_override():
    # Override the dependency
    app.dependency_overrides[get_current_user] = override_get_current_user
    
    try:
        # Make request without real auth header
        response = client.get("/users/me")
        assert response.status_code == 200
        assert response.json() == {
            "user": {"id": 99, "username": "test_user", "is_test": True}
        }
    finally:
        # Clean up - remove override
        app.dependency_overrides.clear()


def test_get_profile_without_override():
    """Test without override - should fail without valid token."""
    app.dependency_overrides.clear()
    response = client.get("/users/me")
    # Missing header causes 422 (validation error)
    assert response.status_code == 422
```

#### Using Pytest with Dependency Overrides

A more organized approach using pytest fixtures:

```python
# conftest.py
import pytest
from fastapi.testclient import TestClient
from main import app, get_current_user


@pytest.fixture
def test_user():
    """Test user data."""
    return {"id": 1, "username": "testuser", "email": "test@example.com"}


@pytest.fixture
def mock_get_current_user(test_user):
    """Factory for creating mock auth dependency."""
    def _mock():
        return test_user
    return _mock


@pytest.fixture
def client(mock_get_current_user):
    """Test client with dependencies overridden."""
    app.dependency_overrides[get_current_user] = mock_get_current_user
    
    with TestClient(app) as test_client:
        yield test_client
    
    # Clean up after tests
    app.dependency_overrides.clear()
```

```python
# test_api.py
def test_get_profile(client, test_user):
    """Test profile endpoint with mocked authentication."""
    response = client.get("/users/me")
    assert response.status_code == 200
    data = response.json()
    assert data["user"]["username"] == test_user["username"]


def test_create_item(client):
    """Test creating an item with mocked auth."""
    response = client.post("/items", json={"name": "Test Item", "price": 10.0})
    assert response.status_code == 201
```

#### Complete Testing Example

Let's build a complete example with a database dependency:

```python
# main.py
from typing import Annotated, Generator
from fastapi import FastAPI, Depends, HTTPException
from pydantic import BaseModel

app = FastAPI()


# Models
class Item(BaseModel):
    id: int
    name: str
    price: float


class ItemCreate(BaseModel):
    name: str
    price: float


# Simulated database
class Database:
    def __init__(self, db_type: str = "production"):
        self.db_type = db_type
        self._items = {}
        self._next_id = 1
    
    def get_items(self):
        return list(self._items.values())
    
    def get_item(self, item_id: int):
        return self._items.get(item_id)
    
    def create_item(self, item: ItemCreate):
        new_item = {"id": self._next_id, **item.model_dump()}
        self._items[self._next_id] = new_item
        self._next_id += 1
        return new_item
    
    def delete_item(self, item_id: int):
        if item_id in self._items:
            del self._items[item_id]
            return True
        return False


def get_db() -> Generator[Database, None, None]:
    """Production database dependency."""
    db = Database("production")
    print(f"Connecting to {db.db_type} database...")
    try:
        yield db
    finally:
        print(f"Closing {db.db_type} database connection...")


DBDep = Annotated[Database, Depends(get_db)]


def get_current_user():
    """Production user dependency."""
    return {"id": 1, "username": "production_user"}


UserDep = Annotated[dict, Depends(get_current_user)]


# Endpoints
@app.get("/items", response_model=list[Item])
async def list_items(db: DBDep):
    return db.get_items()


@app.post("/items", response_model=Item, status_code=201)
async def create_item(item: ItemCreate, db: DBDep, user: UserDep):
    created = db.create_item(item)
    return created


@app.get("/items/{item_id}", response_model=Item)
async def get_item(item_id: int, db: DBDep):
    item = db.get_item(item_id)
    if item is None:
        raise HTTPException(status_code=404, detail="Item not found")
    return item


@app.delete("/items/{item_id}")
async def delete_item(item_id: int, db: DBDep, user: UserDep):
    if db.delete_item(item_id):
        return {"deleted": True}
    raise HTTPException(status_code=404, detail="Item not found")


@app.get("/users/me")
async def get_profile(user: UserDep):
    return {"user": user}
```

```python
# test_main.py
import pytest
from fastapi.testclient import TestClient
from main import app, get_db, get_current_user, Database


# Test fixtures
@pytest.fixture
def test_db():
    """Test database with initial data."""
    db = Database("test")
    db._items = {
        1: {"id": 1, "name": "Existing Item", "price": 50.0},
        2: {"id": 2, "name": "Another Item", "price": 25.0},
    }
    db._next_id = 3
    return db


@pytest.fixture
def test_user():
    """Test user."""
    return {"id": 99, "username": "test_user", "role": "admin"}


@pytest.fixture
def override_get_db(test_db):
    """Factory for database override."""
    def _override():
        yield test_db
    return _override


@pytest.fixture
def override_get_user(test_user):
    """Factory for user override."""
    def _override():
        return test_user
    return _override


@pytest.fixture
def client(override_get_db, override_get_user):
    """Test client with all dependencies overridden."""
    app.dependency_overrides[get_db] = override_get_db
    app.dependency_overrides[get_current_user] = override_get_user
    
    with TestClient(app) as test_client:
        yield test_client
    
    app.dependency_overrides.clear()


# Tests
class TestItemList:
    def test_list_items(self, client):
        """Test listing items."""
        response = client.get("/items")
        assert response.status_code == 200
        data = response.json()
        assert len(data) == 2
        assert data[0]["name"] == "Existing Item"


class TestItemCreate:
    def test_create_item(self, client, test_user):
        """Test creating an item."""
        response = client.post(
            "/items",
            json={"name": "New Item", "price": 100.0},
        )
        assert response.status_code == 201
        data = response.json()
        assert data["name"] == "New Item"
        assert data["price"] == 100.0
        assert data["id"] == 3  # Next ID in test db


class TestItemGet:
    def test_get_existing_item(self, client):
        """Test getting an existing item."""
        response = client.get("/items/1")
        assert response.status_code == 200
        data = response.json()
        assert data["id"] == 1
        assert data["name"] == "Existing Item"

    def test_get_nonexistent_item(self, client):
        """Test getting a nonexistent item returns 404."""
        response = client.get("/items/999")
        assert response.status_code == 404


class TestItemDelete:
    def test_delete_item(self, client, test_db):
        """Test deleting an item."""
        response = client.delete("/items/1")
        assert response.status_code == 200
        assert response.json()["deleted"] is True
        # Verify it's deleted
        assert test_db.get_item(1) is None

    def test_delete_nonexistent_item(self, client):
        """Test deleting a nonexistent item returns 404."""
        response = client.delete("/items/999")
        assert response.status_code == 404


class TestUserProfile:
    def test_get_profile(self, client, test_user):
        """Test getting user profile."""
        response = client.get("/users/me")
        assert response.status_code == 200
        data = response.json()
        assert data["user"]["username"] == test_user["username"]
```

#### Running Tests with Different Scenarios

```python
# test_scenarios.py
import pytest
from fastapi.testclient import TestClient
from fastapi import HTTPException
from main import app, get_db, get_current_user, Database


@pytest.fixture
def client_with_empty_db():
    """Client with empty database for testing edge cases."""
    def empty_db():
        yield Database("empty_test")
    
    def test_user():
        return {"id": 1, "username": "tester"}
    
    app.dependency_overrides[get_db] = empty_db
    app.dependency_overrides[get_current_user] = test_user
    
    with TestClient(app) as client:
        yield client
    
    app.dependency_overrides.clear()


def test_empty_item_list(client_with_empty_db):
    """Test listing items when database is empty."""
    response = client_with_empty_db.get("/items")
    assert response.status_code == 200
    assert response.json() == []


def test_error_simulation():
    """Test simulating a dependency error."""
    def failing_db():
        raise HTTPException(status_code=503, detail="Database unavailable")
        yield  # Never reached
    
    app.dependency_overrides[get_db] = failing_db
    
    with TestClient(app) as client:
        response = client.get("/items")
        assert response.status_code == 503
        assert "unavailable" in response.json()["detail"]
    
    app.dependency_overrides.clear()
```

#### Override for Specific Tests Only

```python
# test_specific_overrides.py
import pytest
from fastapi.testclient import TestClient
from main import app, get_current_user


@pytest.fixture
def client():
    """Client without overrides - use for real auth tests."""
    return TestClient(app)


@pytest.fixture
def client_as_admin():
    """Client with admin user override."""
    def admin_user():
        return {"id": 1, "username": "admin", "role": "admin"}
    
    app.dependency_overrides[get_current_user] = admin_user
    
    with TestClient(app) as client:
        yield client
    
    app.dependency_overrides.clear()


@pytest.fixture
def client_as_regular_user():
    """Client with regular user override."""
    def regular_user():
        return {"id": 2, "username": "regular", "role": "user"}
    
    app.dependency_overrides[get_current_user] = regular_user
    
    with TestClient(app) as client:
        yield client
    
    app.dependency_overrides.clear()


def test_as_admin(client_as_admin):
    """Test with admin privileges."""
    response = client_as_admin.get("/users/me")
    assert response.json()["user"]["role"] == "admin"


def test_as_regular_user(client_as_regular_user):
    """Test with regular user privileges."""
    response = client_as_regular_user.get("/users/me")
    assert response.json()["user"]["role"] == "user"
```

---

### Summary

In this chapter, you've mastered FastAPI's dependency injection system:

1. **The `Depends` Function**: Creating reusable dependencies for common logic like authentication, pagination, and database sessions.

2. **Dependency Trees**: Building hierarchical dependencies where dependencies depend on other dependencies, creating clean separation of concerns.

3. **Yield Dependencies**: Managing resources with setup and teardown logic using generators, perfect for database connections and file handles.

4. **Dependency Overrides**: Replacing dependencies during testing for isolated, fast, and reliable tests.

---

### Exercises

1. **Pagination Dependency**: Create a `Pagination` dependency class that:
   - Accepts `page` and `page_size` query parameters
   - Calculates `skip` and `limit` automatically
   - Validates `page_size` is between 1 and 100
   - Returns a dictionary with pagination info

2. **Authentication Chain**: Build an authentication dependency tree with:
   - `get_token`: Extracts token from header
   - `decode_token`: Validates JWT and returns payload
   - `get_user`: Fetches user from database
   - `require_active`: Ensures user is active
   - `require_admin`: Ensures user is admin (optional)

3. **Database Session**: Create a yield dependency that:
   - Connects to a simulated database
   - Handles transactions (commit on success, rollback on error)
   - Always closes the connection in `finally`
   - Logs all operations

4. **Testing with Overrides**: Write tests for an API with:
   - A database dependency overridden with in-memory storage
   - An authentication dependency overridden with test users
   - Tests for success and error scenarios
   - Cleanup that removes all overrides

---

### What's Next?

**Chapter 7: Request Handling and Context** will explore how to work directly with HTTP requests:
- Accessing the `Request` object for headers, cookies, and client info
- Handling form data and file uploads in detail
- Working with cookies for session management
- Manipulating HTTP headers in requests and responses



<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../2. data_validation_and_serialization_with_pydantic/5. advanced_data_handling.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='7. request_handling_and_context.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
